In [1]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import logging, os, glob
from _logging import set_logging
from _metrics import display_metrics
from _pckle import save_pickle_object, load_pickle_object
from _utility import gl, get_perc, get_dictionaries_from_list
from _model import train_model
from _graph import accuracy_loss_graph

set_logging(logging)
logging.info(f"Cuda is available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes = ["Business", "Other"]
dict_classes, dict_classes_rev = get_dictionaries_from_list(classes)


2023-02-06 08:38:20,127 | INFO : Cuda is available: True


In [2]:
train_loader = load_pickle_object(gl.pkl_train_loader)
val_loader = load_pickle_object(gl.pkl_val_loader)
test_loader = load_pickle_object(gl.pkl_test_loader)

2023-02-06 08:38:20,219 | INFO : Loading pickle file from: pickle\pkle_train_loader.pkl
2023-02-06 08:38:20,826 | INFO : Loading pickle file from: pickle\pkle_val_loader.pkl
2023-02-06 08:38:20,904 | INFO : Loading pickle file from: pickle\pkle_test_loader.pkl


In [3]:
def imshow(inp, _mean, _std, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array(_mean)
    std = np.array(_std)
    inp = std * inp + mean  # denormalise
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [4]:
def visualize_model(model, dataloaders, classes, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {classes[preds[j]]}')
                imshow(inputs.cpu().data[j])
                if images_so_far == num_images:
                    model.train(mode=was_training)
                return
        model.train(mode=was_training)

In [5]:
dataloaders = {"train": train_loader, "val": val_loader}
train_dataset_size = len(train_loader.dataset)
val_dataset_size = len(val_loader.dataset)
dataset_sizes = {"train": train_dataset_size, "val": val_dataset_size}
logging.info(f"Train dataset size: {train_dataset_size}")
logging.info(f"Val dataset size: {val_dataset_size}")
# Get the latest version of the Resnet weights and freeze the layers
model_conv = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
for param in model_conv.parameters():
    param.requires_grad = False
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)
model_conv = model_conv.to(device)
criterion = nn.CrossEntropyLoss()
# Only parameters of final layer are being optimized
optimizer_ft = optim.Adam(model_conv.fc.parameters(), lr=0.0001)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

2023-02-06 08:38:21,114 | INFO : Train dataset size: 35046
2023-02-06 08:38:21,115 | INFO : Val dataset size: 8762


## Set up Checkpoint and Early Stopping

In [6]:
if os.path.exists(gl.chk_dir) == False:
    os.mkdir(gl.chk_dir)
    
checkpoint_path = os.path.join(gl.chk_dir, gl.chk_resnet_model)

In [7]:
def load_checkpoint(checkpoint_path, model_conv, optimizer_ft):
    checkpoint = torch.load(checkpoint_path)
    model_conv.load_state_dict(checkpoint["model_state_dict"])
    optimizer_ft.load_state_dict(checkpoint["model_state_dict"])
    epoch = checkpoint["epoch"]
    loss = checkpoint["loss"]
    return checkpoint, model_conv, optimizer_ft, epoch, loss

In [8]:
load_checkpoint_flag = False
if load_checkpoint_flag:
    checkpoint, model_conv, optimizer_ft, epoch, loss = \
        load_checkpoint(checkpoint_path, model_conv, optimizer_ft)
    
patience = 2
model_conv = train_model(model_conv, logging, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, dataset_sizes, 
                         checkpoint_path, patience, num_epochs=25)

save_pickle_object(model_conv, gl.pkl_model_conv)

2023-02-06 08:38:23,201 | INFO : Epoch 1/25
2023-02-06 08:38:23,202 | INFO : ----------


d:\Anaconda\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2023-02-06 09:26:47,423 | INFO : train Loss: 0.5771 Acc: 0.6934
2023-02-06 09:39:06,412 | INFO : val Loss: 0.5213 Acc: 0.7404
Best validation accuracy: 0.7403560830860534
Saving best model for epoch: 1
2023-02-06 09:39:09,069 | INFO : Epoch 2/25
2023-02-06 09:39:09,070 | INFO : ----------
2023-02-06 10:19:31,413 | INFO : train Loss: 0.5116 Acc: 0.7492
2023-02-06 10:36:29,096 | INFO : val Loss: 0.4978 Acc: 0.7598
EarlyStopping counter: 1 out of 2
2023-02-06 10:36:29,787 | INFO : Early stopping for epoch: 2
2023-02-06 10:36:29,788 | INFO : Training complete in 118m 7s
2023-02-06 10:36:29,788 | INFO : Best val Acc: 0.000000
2023-02-06 10:36:31,009 | INFO : Saving pickle file from: pickle\model_conv.pkl


In [9]:
visualize_model(model_conv, dataloaders, classes)

plt.ioff()
plt.show()

AttributeError: 'dict' object has no attribute 'training'